# S&P500 Example: Hedging of an Asian Put Option, Extended Testing Window

First import the necessary packages

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from Functions import *
from scipy.optimize import minimize

Only execute next box if program should run on a GPU

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

Load the Data of 20 of the largest constituents

In [3]:
stocks =[]
stocks += [pd.read_csv("time_series/AAPL.OQ.csv")] #Apple Inc.
stocks += [pd.read_csv("time_series/MSFT.OQ.csv")] # Microsoft Corporation 
stocks += [pd.read_csv("time_series/AMZN.OQ.csv")] # Amazon.com Inc.
stocks += [pd.read_csv("time_series/GOOGL.OQ.csv")] #Alphabet Inc. Class C
stocks += [pd.read_csv("time_series/BRKb.N.csv")] # Berkshire Hathaway Inc. Class B
stocks += [pd.read_csv("time_series/JPM.N.csv")] #JPMorgan Chase & Co.
stocks += [pd.read_csv("time_series/JNJ.N.csv")] #Johnson & Johnsonjohn
stocks += [pd.read_csv("time_series/V.N.csv")] # Visa Inc. Class A
stocks += [pd.read_csv("time_series/UNH.N.csv")] #UnitedHealth Group Incorporated
stocks += [pd.read_csv("time_series/NVDA.OQ.csv")] #NVIDIA Corporation
stocks += [pd.read_csv("time_series/PG.N.csv")] #Procter & Gamble Company
stocks += [pd.read_csv("time_series/HD.N.csv")] #Home Depot Inc.
stocks += [pd.read_csv("time_series/MA.N.csv")] # Mastercard Incorporated Class A
stocks += [pd.read_csv("time_series/BAC.N.csv")] # Bank of America Corp
stocks += [pd.read_csv("time_series/DIS.N.csv")] # Walt Disney Company
stocks += [pd.read_csv("time_series/CMCSA.OQ.csv")] #Comcast Corporation Class A
stocks += [pd.read_csv("time_series/XOM.N.csv")] #Exxon Mobil Corporation
stocks += [pd.read_csv("time_series/ADBE.OQ.csv")] # Adobe Inc.
stocks += [pd.read_csv("time_series/VZ.N.csv")] #Verizon Communications Inc.
stocks += [pd.read_csv("time_series/INTC.OQ.csv")] # Intel Corporation


for i in range(len(stocks)):
    # Clean the NaNs
    stocks[i] = stocks[i].dropna()
    # Reset Indices
    stocks[i] = stocks[i].reset_index()
    # Only Dates and Close Values
    stocks[i]  = stocks[i] [["Date","CLOSE"]]

# Maxmimum Likelihood Estimation for all constituents

Define the function for the estimation (Can choose between stochastic gradient descent, gradient descent, and ADAM Optimizer)

In [4]:
def compute_max_parameters(x, 
                           iterations = 1000,
                           initial_guess= [0.1,0.1,0,0,0.75],
                           tolerance = 1e-15,
                           Delta = 1/250.,
                           method = 'COBYLA'):
    x_0 = np.array(initial_guess) #Initial guess
    eps = tolerance # Tolerance to avoid that fractions and log-expressions become inf or -inf
    
    #Definte the Log-Likelihood Function
    def log_likelihood(param):
        a_0 = param[0]
        a_1 = param[1]
        b_0 = param[2]
        b_1 = param[3]
        gamma = param[4]
        constant = np.sqrt(2*np.pi*Delta)
        l= [-np.log((a_0+a_1*np.maximum(x[i],0))**gamma*constant+eps)-(1/(2*Delta))*((x[i+1]-x[i]-(b_0+b_1*x[i])*Delta)/(a_0+a_1*np.maximum(x[i],0)+eps)**gamma)**2 for i in range(len(x)-1)]
        return -np.mean(l) # Mean instead of sum to have smaller values
    
    a0,a1,b0,b1,gamma = minimize(log_likelihood,x_0,method=method,options={'maxiter': iterations,
                                                                          'rhobeg':0.01},
                bounds = [(eps,None),(eps,None),(None,None),(None,None),(eps,None)]).x
    a0 = np.max([a0,eps])
    a1 = np.max([a1,eps])
    gamma = np.min([np.max([gamma,eps]),1.2]) # artificial lower/upper bound
    return a0,a1,b0,b1,gamma

Determination of MLE-Parameters

In [5]:
start_index = 2880
end_index = 2910
list_parameters_extended=[]

for j in range(len(stocks)):
    print("MLE for stock {}".format((str(j))))
    #Create lists for the parameters
    list_a0 = []
    list_a1 = []
    list_b0 = []
    list_b1 = []
    list_gamma = []
    list_a0_rescaled = []
    list_a1_rescaled = []
    list_b0_rescaled = []
    list_b1_rescaled = []
    
    #Compute optimal parameters
    for i in np.arange(2979,3275,100): # the additional periods
        x = np.array(stocks[j].iloc[(i-250):i]["CLOSE"])
        a0,a1,b0,b1,gamma = compute_max_parameters(x)
        
        list_a0 += [a0]
        list_a1 += [a1]
        list_b0 += [b0]
        list_b1 += [b1]
        list_gamma += [gamma]
        # Rescale the parameters
    list_parameters_extended+= [pd.DataFrame({'a0': list_a0,
                   'a1': list_a1,
                   'b0': list_b0,
                   'b1': list_b1,
                   'gamma': list_gamma})]
    list_parameters_extended[j].to_csv('csv/mle_parameters_stock'+str(j)+"_extended_period"+'.csv',index=False)

MLE for stock 0


C:\Users\julia\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\scipy\optimize\_minimize.py:540: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)


MLE for stock 1
MLE for stock 2
MLE for stock 3
MLE for stock 4
MLE for stock 5
MLE for stock 6
MLE for stock 7
MLE for stock 8
MLE for stock 9
MLE for stock 10
MLE for stock 11
MLE for stock 12
MLE for stock 13
MLE for stock 14
MLE for stock 15
MLE for stock 16
MLE for stock 17
MLE for stock 18
MLE for stock 19


In [6]:
list_parameters_extended=[]
for j in range(len(stocks)):
     list_parameters_extended+= [pd.read_csv('csv/mle_parameters_stock'+str(j)+"_extended_period"+'.csv')]

Load the other estimated parameters.

In [7]:
list_parameters=[]
for j in range(len(stocks)):
     list_parameters+= [pd.read_csv('csv/mle_parameters_stock'+str(j)+'.csv')]

# Testing not only in a single period, but in testing window

We do the same procedure, and train the strategy, but evaluate this time every 30 trading days until September 2021. (13 Periods)

In [8]:
# Create Lists to collect the hedging errors
derivative_payoff = []
error_fix = []
error_robust = []
rel_error_fix = []
rel_error_robust = []
epochs = 1000 # Number of epochs for training

In [9]:
for i in range(1,4):
    print("Period : {}".format(i))
    for j in range(len(stocks)):        
        print("\n\n ############\n \n ##### Calculations for Stock: {} \n".format(str(j+1)))
        # Load Parameters from the List
        list_gamma = list_parameters[j]['gamma']
        list_a0 = list_parameters[j]['a0']
        list_a1 = list_parameters[j]['a1']
        list_b0 = list_parameters[j]['b0']
        list_b1 = list_parameters[j]['b1']
        
#         # Extend the parameters
        list_gamma = list_gamma.append(list_parameters_extended[j]['gamma'][:i])
        list_a0 = list_a0.append(list_parameters_extended[j]['a0'][:i])
        list_a1 = list_a1.append(list_parameters_extended[j]['a1'][:i])
        list_b0 = list_b0.append(list_parameters_extended[j]['b0'][:i])
        list_b1 = list_b1.append(list_parameters_extended[j]['b1'][:i])
        
        # Treatings Nans
        list_a0 = list_a0.fillna(list_a0.mean()) 
        list_a1 = list_a1.fillna(list_a1.mean()) 
        list_b0 = list_b0.fillna(list_b0.mean()) 
        list_b1 = list_b1.fillna(list_b1.mean()) 
        list_gamma = list_gamma.fillna(list_gamma.mean()) 

        
        start_index = 2880+100*i
        end_index = 2910+100*i
        x_0 = stocks[j].iloc[start_index]["CLOSE"].astype('float32')
        T = (30/250)
        n = 30
        def derivative(path):
            return tf.nn.relu(10-tf.math.reduce_mean(path))
        #######################################################
        #ROBUST
        # Assign the intervals
        a_0=[(list_a0).min(),(list_a0).max()]
        a_1=[(list_a1).min(),(list_a1).max()]
        b_0=[(list_b0).min(),(list_b0).max()]
        b_1=[(list_b1).min(),(list_b1).max()]
        gamma=[(list_gamma).min(),(list_gamma).max()]
        
        # Assign the closing data of the beginning of the test period
        price_robust, hedge_robust = optimal_hedge(derivative,a_0,a_1,b_0,b_1,gamma,x_0,T,n,
                                     depth = 4,nr_neurons = 256,EPOCHS = epochs,l_r = 0.005,BATCH_SIZE =256,scaling_factor=x_0/10.)
        #######################################################
        # NON ROBUST
        a_0_fix=[list_parameters_extended[j]['a0'][i-1],list_parameters_extended[j]['a0'][i-1]]
        a_1_fix=[list_parameters_extended[j]['a1'][i-1],list_parameters_extended[j]['a1'][i-1]]
        b_0_fix=[list_parameters_extended[j]['b0'][i-1],list_parameters_extended[j]['b0'][i-1]]
        b_1_fix=[list_parameters_extended[j]['b1'][i-1],list_parameters_extended[j]['b1'][i-1]] 
        gamma_fix=[list_parameters_extended[j]['gamma'][i-1],list_parameters_extended[j]['gamma'][i-1]] 
        price_fix, hedge_fix = optimal_hedge(derivative,a_0_fix,a_1_fix,b_0_fix,b_1_fix,gamma_fix,x_0,T,n,
                                 depth = 4,nr_neurons = 256,EPOCHS = epochs,l_r = 0.005,BATCH_SIZE =256,scaling_factor=x_0/10.)



        ####################################################
        #Evaluate
        t_k = np.linspace(0,T,n+1)
        x_0 = stocks[j].iloc[start_index]["CLOSE"].item()
        test_period= np.array(stocks[j].iloc[start_index:end_index]["CLOSE"])/(x_0/10.) #Normalize the values of the test period

        # Fix Parameters
        hedge_evaluated_fix = [hedge_fix([tf.reshape(test_period[i],(1,1)),tf.reshape(t_k[i],(1,1))]) for i in range(n-1)]
        delta_S_fix = tf.reduce_sum([hedge_evaluated_fix[i]*np.reshape(np.diff(test_period)[i],(1,1)) for i in range(n-1)],0)

        # Robust Parameters
        hedge_evaluated_robust = [hedge_robust([tf.reshape(test_period[i],(1,1)),tf.reshape(t_k[i],(1,1))]) for i in range(n-1)]
        delta_S_robust = tf.reduce_sum([hedge_evaluated_robust[i]*np.reshape(np.diff(test_period)[i],(1,1)) for i in range(n-1)],0)

        #########################################################
        # Derivative
        derivative_on_batch = derivative(test_period)
        derivative_payoff+= [derivative(test_period)]
        # Compute Hedging Error (Relative), Percentage Values:
        rel_error_fix += [(delta_S_fix.numpy()+price_fix - derivative_on_batch.numpy())/price_fix]
        rel_error_robust += [(delta_S_robust.numpy()+price_robust - derivative_on_batch.numpy())/price_robust]
        error_fix += [10*(delta_S_fix.numpy()+price_fix - derivative_on_batch.numpy())]
        error_robust += [10*(delta_S_robust.numpy()+price_robust - derivative_on_batch.numpy())]

Period : 1


 ############
 
 ##### Calculations for Stock: 1 



  1%|          | 11/1000 [00:06<09:27,  1.74it/s]

Iteration:10, Price of Hedge: 33.559571193365265, Loss: 104.01507220292697


  2%|▏         | 21/1000 [00:11<09:03,  1.80it/s]

Iteration:20, Price of Hedge: 33.31145383315161, Loss: 43.54787849901477


...

 98%|█████████▊| 981/1000 [10:02<00:11,  1.67it/s]

Iteration:980, Price of Hedge: 1.1298850533114775, Loss: 0.03544806489637225


 99%|█████████▉| 991/1000 [10:08<00:05,  1.66it/s]

Iteration:990, Price of Hedge: 1.1355790859007584, Loss: 0.04485896471515787


100%|██████████| 1000/1000 [10:13<00:00,  1.63it/s]


In [10]:
error_fix = [e[0][0] for e in error_fix]
error_robust = [e[0][0] for e in error_robust]
rel_error_fix = [e[0][0] for e in rel_error_fix]
rel_error_robust = [e[0][0] for e in rel_error_robust]

In [11]:
list_computations= pd.DataFrame({'error_fix': error_fix,
                   'error_robust': error_robust})
list_computations.to_csv('csv/sp500_results_asian_atm_rolling_window.csv',index=False)
list_computations_rel= pd.DataFrame({'error_fix': rel_error_fix,
                   'error_robust': rel_error_robust})
list_computations_rel.to_csv('csv/sp500_results_asian_atm_rolling_window_relative.csv',index=False)

Absolute Error

In [12]:
np.abs(list_computations).describe()

,error_fix,error_robust
count,60.000000,60.000000
mean,1.306456,2.149686
std,0.782881,2.829966
min,0.086689,0.172926
25%,0.723576,0.626821
50%,1.128658,1.025174
75%,1.921974,1.932394
max,3.054011,12.630735


Relative Error

In [13]:
np.abs(list_computations_rel).describe()

,error_fix,error_robust
count,60.000000,60.000000
mean,0.442418,0.556593
std,0.227262,0.438035
min,0.026533,0.078179
25%,0.258070,0.318031
50%,0.420156,0.416660
75%,0.600333,0.751668
max,0.960834,2.915202
